<a href="https://colab.research.google.com/github/harsh-ux/Traffic-Sign-Recognition/blob/main/MicroNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
 
uploaded = files.upload()
 
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

 97% 594M/612M [00:06<00:00, 84.5MB/s]
100% 612M/612M [00:06<00:00, 102MB/s] 


In [ ]:
import zipfile
x = zipfile.ZipFile('/content/gtsrb-german-traffic-sign.zip')
x.extractall('/content/')

In [ ]:
import os
os.remove("/content/gtsrb-german-traffic-sign.zip")
print("File Removed!")

File Removed!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
 
# once the images are loaded, how do we pre-process them before being passed into the network
# by default, we resize the images to 32 x 32 in size
# and normalize them to mean = 0 and standard-deviation = 1 based on statistics collected from
# the training set

data_transforms = transforms.Compose([
    transforms.Resize((48, 48)),
    #transforms.Grayscale(num_output_channels=1),
    transforms.GaussianBlur(5),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
])

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets
 
#df_test = pd.read_csv('/content/Test.csv')
#df_train = pd.read_csv('/content/Train.csv')
#train_path = df_train.iloc[:, -1]
#test_path = df_test.iloc[:, -1]
#print(list(train_path))
 
dataset = ImageFolder('/content/Train', data_transforms)
print(len(dataset))
datasets = train_val_dataset(dataset)
print(len(datasets['train']))
print(len(datasets['val']))
dataloaders = {x:DataLoader(datasets[x],32, shuffle=True, num_workers=4) for x in ['train','val']}
x,y = next(iter(dataloaders['train']))
print(x.shape, y.shape)

39209
29406
9803


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([32, 3, 48, 48]) torch.Size([32])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
 
nclasses = 43 # GTSRB as 43 classes
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=1)
        self.conv2 = nn.Conv2d(1, 29, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv3 = nn.Conv2d(29, 59, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv4 = nn.Conv2d(59, 74, kernel_size=3)
        self.maxpool4 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1184, 300)
        self.fc2 = nn.Linear(300, nclasses)
        self.conv0_bn = nn.BatchNorm2d(3)
        self.conv1_bn = nn.BatchNorm2d(1)
        self.conv2_bn = nn.BatchNorm2d(29)
        self.conv3_bn = nn.BatchNorm2d(59)
        self.conv4_bn = nn.BatchNorm2d(74)
        self.dense1_bn = nn.BatchNorm1d(300)
    def forward(self, x):
        x =  F.relu(self.conv1_bn(self.conv1(self.conv0_bn(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3( self.maxpool2(x))))
        x = F.relu(self.conv4_bn(self.conv4( self.maxpool3(x))))
        x = self.maxpool4(x)        
        x = x.view(-1, 1184)
        x = F.relu(self.fc1(x))
        x = self.dense1_bn(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x)

In [ ]:
from __future__ import print_function
import argparse
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
 
batch_size =50                                                                                                                                                                                     
epochs = 10                                                                                                                                                                                                                                                                                                                                                                                 
seed = 1                                                                                                                                                                                           
log_interval=180                                                                                                                                                                                   
data = "data"                                                                                                                                                                                                                                                                                                                                                                                          
torch.manual_seed(1)                                                                                                                                                                               
lr =0.007                                                                                                                                                                                          
momentum = 0.8                                                                                                                                                                                     
decay = 0.9                                                                                                                                                                                        
step = 1000                                                                                                                                                                                        
l2_norm = 0.00001  
cuda = True
resume = False
# These may change as described in paper

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net()
if  cuda: 
    model.to(device)
 
if resume :
    state_dict = torch.load("model_28.pth")
    model.load_state_dict(state_dict)

In [ ]:
def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in dataloaders['val']:
        data, target = Variable(data, volatile=True), Variable(target)
        data =data.to(device)
        target =target.to(device)
        output = model(data)
        validation_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    validation_loss /= len(dataloaders['val'].dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.5f}%)\n'.format(
        validation_loss, correct, len(dataloaders['val'].dataset),
        100. * correct / len(dataloaders['val'].dataset)))
    return validation_loss
 
def train(epoch , train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(dataloaders['train']):
        data, target = Variable(data), Variable(target)
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target).cuda()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloaders['train'].dataset),
                100. * batch_idx / len(dataloaders['train']), loss.item()))

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr , momentum=momentum, weight_decay=l2_norm, nesterov=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay)
 
temp = 10
for epoch in range(1, epochs+1):
    train(epoch, dataloaders['train'])
    val = validation()
    if epoch % step :
        scheduler.step()
    if val < temp : 
        temp = val
        model_file = 'model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py ' + model_file + '` to generate the Kaggle formatted csv file')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/29406 (0%)]	Loss: 1.611350
Train Epoch: 1 [5760/29406 (20%)]	Loss: 1.817069
Train Epoch: 1 [11520/29406 (39%)]	Loss: 1.143993
Train Epoch: 1 [17280/29406 (59%)]	Loss: 1.787257
Train Epoch: 1 [23040/29406 (78%)]	Loss: 1.843669
Train Epoch: 1 [28800/29406 (98%)]	Loss: 1.996617


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Validation set: Average loss: 0.3896, Accuracy: 9389/9803 (95.77680%)


Saved model to model_1.pth. You can run `python evaluate.py model_1.pth` to generate the Kaggle formatted csv file
Train Epoch: 2 [0/29406 (0%)]	Loss: 1.925807
Train Epoch: 2 [5760/29406 (20%)]	Loss: 1.378313
Train Epoch: 2 [11520/29406 (39%)]	Loss: 2.518625
Train Epoch: 2 [17280/29406 (59%)]	Loss: 1.825117
Train Epoch: 2 [23040/29406 (78%)]	Loss: 1.682228
Train Epoch: 2 [28800/29406 (98%)]	Loss: 1.835055

Validation set: Average loss: 0.1728, Accuracy: 9750/9803 (99.45935%)


Saved model to model_2.pth. You can run `python evaluate.py model_2.pth` to generate the Kaggle formatted csv file
Train Epoch: 3 [0/29406 (0%)]	Loss: 1.779407
Train Epoch: 3 [5760/29406 (20%)]	Loss: 1.826796
Train Epoch: 3 [11520/29406 (39%)]	Loss: 1.457605
Train Epoch: 3 [17280/29406 (59%)]	Loss: 1.687416
Train Epoch: 3 [23040/29406 (78%)]	Loss: 1.413066
Train Epoch: 3 [28800/29406 (98%)]	Loss: 2.054653

Validation set: Average loss: 0.143